In [626]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
import time
load_dotenv()

True

In [627]:
data = []
base_url = "https://facebook.com/"

In [ ]:
def get_website(url = base_url):
    chrome_options = webdriver.ChromeOptions()
    # Tat thong bao (alert) va mo full size cua so
    prefs = {
    "profile.default_content_setting_values.notifications": 2,    # 1: Cho phép, 2: Chặn
    "profile.default_content_setting_values.geolocation": 2,     # Chặn hỏi vị trí
    "profile.default_content_setting_values.media_stream": 2,    # Chặn hỏi Camera/Mic
    "profile.managed_default_content_settings.images": 1,        # 2 nếu bạn muốn tắt cả load ảnh để chạy nhanh hơn
    "credentials_enable_service": False,                         # Tắt hỏi lưu mật khẩu
    "profile.password_manager_enabled": False                    # Tắt trình quản lý mật khẩu
    }

    chrome_options.add_experimental_option("prefs",prefs)
    chrome_options.add_argument("--force-device-scale-factor=0.6")
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()

    # Truy cập vào facebook.com và tìm form đăng nhập, thông tin từ .env
    driver.get(url)
    login_form = driver.find_element(By.CSS_SELECTOR, "form[data-testid='royal_login_form']")
    username = login_form.find_element(By.CSS_SELECTOR,"input[data-testid='royal-email']")
    password = login_form.find_element(By.CSS_SELECTOR,"input[data-testid='royal-pass']")
    your_username = os.getenv("USERRNAME")
    your_password = os.getenv("PASSWORD")
    username.send_keys(your_username)
    password.send_keys(your_password)
    
    # Kiếm nút đăng nhập để bấm
    login_btn = login_form.find_element(By.CSS_SELECTOR,"button[data-testid='royal-login-button']")
    login_btn.click()

    print("Hiện đang giải Captcha...")
    try:
        wait = WebDriverWait(driver,60)
        search_bar = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[aria-label='Tìm kiếm trên Facebook']")))
        print("Đã nhận diện được search_bar")
    except Exception:
        import traceback
        print("--- PHÁT HIỆN LỖI THẬT SỰ TẠI ĐÂY ---")
        traceback.print_exc() 
        print("------------------------------------")
    
    time.sleep(10)

    return driver       

In [692]:
test_driver = get_website()

Hiện đang giải Captcha...
Đã nhận diện được search_bar


In [630]:
# Nút để mở bình luận
def open_post(driver):
    try:
        comment_btn = driver.find_element(By.CSS_SELECTOR,"div[data-ad-rendering-role='comment_button']")
        # Phải click bằng js vì selenium đã bị chặn bởi các thủ thuật như hiển thị size 0x0 
        test_driver.execute_script("arguments[0].click();", comment_btn)
    except:
        print("Không thấy post để mở!")

In [639]:
open_post(test_driver)

In [631]:
# Nút để đóng bình luận
def close_post(driver):
    try:
        close_btn = driver.find_element(By.CSS_SELECTOR,"div[aria-label='Đóng']")
        driver.execute_script("arguments[0].click();",close_btn)
    except:
        print("Chưa có bài viết được mở ra!")

In [638]:
close_post(test_driver)

In [466]:
comment_btns = test_driver.find_elements(By.CSS_SELECTOR,"div[data-ad-rendering-role='comment_button']")
print(comment_btns)

[<selenium.webdriver.remote.webelement.WebElement (session="a345ed17ce42cdf1bd53cecd40944731", element="f.8CB8314FA47A7DAD76B7B88EA7D42073.d.18FE98FFCE72C1E8AC59AEEFBDACFE6F.e.377")>, <selenium.webdriver.remote.webelement.WebElement (session="a345ed17ce42cdf1bd53cecd40944731", element="f.8CB8314FA47A7DAD76B7B88EA7D42073.d.18FE98FFCE72C1E8AC59AEEFBDACFE6F.e.415")>, <selenium.webdriver.remote.webelement.WebElement (session="a345ed17ce42cdf1bd53cecd40944731", element="f.8CB8314FA47A7DAD76B7B88EA7D42073.d.18FE98FFCE72C1E8AC59AEEFBDACFE6F.e.416")>]


In [713]:
def clean_post(driver):
    delete_script = """
        // 1. Tìm và xóa các vùng nội dung chính (story message)
        const postElements = document.querySelectorAll('[data-ad-rendering-role="story_message"], [data-ad-rendering-role="post_message"]');
        postElements.forEach(el => el.remove());

        // 2. Tìm và xóa TẤT CẢ các thẻ <a> (Link, tag tên, hashtag...)
        // Bạn có thể giới hạn phạm vi trong Modal bằng cách dùng: document.querySelectorAll('div[role="dialog"] a')
        const links = document.querySelectorAll('a');
        links.forEach(a => a.remove());

        // 3. Tìm và xóa các ảnh bài viết to
        const images = document.querySelectorAll('div.x1y1aw1k img, .x1pdlv7q img');
        images.forEach(img => {
            if(img.height > 100) {
                img.closest('div.x1n2onr6')?.remove();
            }
        });
    """
    try:
        driver.execute_script(delete_script)
        print("Done: Đã dọn sạch nội dung văn bản, ảnh và toàn bộ link.")
    except Exception as e:
        print(f"Lỗi JS: {e}")

In [714]:
clean_post(test_driver)

Done: Đã dọn sạch nội dung văn bản, ảnh và toàn bộ link.


In [ ]:
def get_clean_comment(driver, comment_element):
    # Script này lọc bỏ các node con là Element (a, span, img) 
    # và chỉ trả về các Text thuần
    js_script = """
    var parent = arguments[0];
    var text = "";
    for (var i = 0; i < parent.childNodes.length; i++) {
        var node = parent.childNodes[i];
        // nodeType === 3 là Text Node (Chữ thuần)
        if (node.nodeType === 3) {
            text += node.textContent;
        }
    }
    return text.trim();
    """
    return driver.execute_script(js_script, comment_element)

In [741]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
# Tìm tất cả các bài viết
def scroll_main(driver,max_scroll=20):
    dataset = set()
    scroll_cnt = 0
    while(scroll_cnt < max_scroll):
        # messages = driver.find_elements(By.CSS_SELECTOR, "div[data-visualcompletion='ignore-dynamic']")
        # for msg in messages:
        #     post_cmt = msg.find_element(By.CSS_SELECTOR,"div[data-ad-rendering-role='comment_button']")
            
        driver.execute_script("window.scrollBy(0, 1000);")
        scroll_cnt+=1
        if(scroll_cnt % 5 == 0):
            time.sleep(2)
    #print(f"Tổng cộng đã quét được: {len(dataset)} bài viết.")

def scroll_post(driver, max_scroll=200, patience=10):
    dataset = set()
    scroll_cnt = 0
    no_change_cnt = 0

    container = driver.find_element(By.CSS_SELECTOR, "div[role='dialog'], .x1n2onr6")
    actions = ActionChains(driver)
    actions.move_to_element_with_offset(container, 50, 200).click().perform()
    last_height = driver.execute_script("return arguments[0].scrollHeight", container)
    
    while scroll_cnt < max_scroll:
        # --- BẮT ĐẦU THU THẬP DỮ LIỆU ---
        current_comments = driver.find_elements(By.CSS_SELECTOR, "div[role='article']")
        previous_size = len(dataset)
        
        for cmt in current_comments:
            try:
                # Tìm vùng nội dung và text thuần
                comment_content_area = cmt.find_element(By.CSS_SELECTOR, ".x1lliihq.xjkvuk6.x1iorvi4")
                comment_div = comment_content_area.find_element(By.CSS_SELECTOR, "div[dir='auto']")
                
                clean_text = get_clean_comment(driver, comment_div)
                if clean_text:
                    dataset.add(clean_text)
            except:
                continue
        
        # Log trạng thái dữ liệu
        if len(dataset) == previous_size:
            no_change_cnt += 1
            print(f"Dữ liệu chưa tăng (Lần {no_change_cnt}/{patience})...")
        else:
            no_change_cnt = 0
            print(f"Đã thu thập được {len(dataset)} comment duy nhất.")
            
        # Điều kiện dừng 1: Theo số patience
        if no_change_cnt >= patience:
            print("Dừng do số lượng comment không tăng thêm.")
            break
            
        # THỰC HIỆN CUỘN 
        actions.move_to_element(container).send_keys(Keys.PAGE_DOWN).perform()
        scroll_cnt += 1
        time.sleep(2)
        
        # Điều kiện dừng 2: Theo chiều cao 
        new_height = driver.execute_script("return arguments[0].scrollHeight", container)
        if new_height == last_height:
            time.sleep(1)
            if new_height == driver.execute_script("return arguments[0].scrollHeight", container):
                print("Đã lướt hết toàn bộ bài viết (scrollHeight không đổi).")
                break
        last_height = new_height
        
    return dataset

In [ ]:
# Hiện các phản hồi Facebook
def display_respond(driver):
    target_xpath = "//span[contains(text(), 'phản hồi') and contains(text(), 'tất cả') and not(contains(text(), 'khác'))]" # Sử dụng xpath để lọc ra các span có text là phản hồi
    try:
        responses = driver.find_elements(By.XPATH, target_xpath)
        while(len(responses) > 0):
            for response in responses:
                driver.execute_script("arguments[0].click();",response)
                time.sleep(2)
            try:
                responses = driver.find_elements(By.XPATH, target_xpath)
            except:
                print("Đã hết phản hồi!")
                responses = 0
    except:
        print("Không có phản hồi nào")

In [550]:
scroll_main(test_driver)

In [743]:
# Demo thử phần tự tìm bài viết và scroll xuống, mở hết các bình luận replied
open_post(test_driver)
time.sleep(3)
clean_post(test_driver)
time.sleep(3)
scroll_post(test_driver)
time.sleep(3)
display_respond(test_driver)

Done: Đã dọn sạch nội dung văn bản, ảnh và toàn bộ link.
Đã thu thập được 38 comment duy nhất.
Đã lướt hết toàn bộ bài viết (scrollHeight không đổi).


In [731]:
unique_comments = set()
comments = test_driver.find_elements(By.CSS_SELECTOR, "div[role='article']")
for cmt in comments:
    try:
        comment_content_area = cmt.find_element(By.CSS_SELECTOR, ".x1lliihq.xjkvuk6.x1iorvi4")
        comment_div = comment_content_area.find_element(By.CSS_SELECTOR, "div[dir='auto']")
        clean_text = get_clean_comment(test_driver, comment_div)
        if clean_text:
            unique_comments.add(clean_text)
            print(f"Đã thêm: {clean_text[:50]}...")
    except:
        continue

Đã thêm: @fan cứng...
Đã thêm: Ngu thì hết nói nổi...
Đã thêm: Tôi thực sự đ hiểu, mặc dù t đ ưa thằng này nhma q...
Đã thêm: cứ sống đơn giản bro nhỉ, thỉnh thoảng cập nhật ch...
Đã thêm: bác muốn xem peter Pan à :))...
Đã thêm: Dkm vừa giỏi, vừa giàu, vừa tài hoa mỗi tội hơi xấ...
Đã thêm: so mặt bằng chung a Hoàng mà xấu thi ko biết đẹp n...
Đã thêm: Cay là do nghĩ được bú 1 mình, ai ngờ phải share v...
Đã thêm: Anh Hoàng trông cũng dễ nhìn, tự lực xây nên cơ ng...
Đã thêm: Hắn ta yêu ai cũng không muốn gọi là vợ, hắn chỉ m...
Đã thêm: Anh Hoàng nói chuẩn nhé, ngày trước anh bảo đồ phụ...
Đã thêm: e này thua xa Ngọc Cream về độ xinh...
Đã thêm: Riêng về anh Hoàng chỉ nhớ ánh mắt anh nhìn Amee t...
Đã thêm: Các ông nói ko sai là ko sai như nào tôi đéo hiểu ...
Đã thêm: Từ lần này đến lần trước a tao goat vl, các mẹ bỉm...
Đã thêm: Thế vụ này năm ngoái kết lại có thật là ông này tá...
Đã thêm: Post a hoàng a hoàng có phủ nhận ko tán e thảo đâu...
Đã thêm: mấy con dẩm cứ cắn a Hoàng t,

In [744]:
print(f"--- THỐNG KÊ ---")
print(f"Tổng số bình luận quét được: {len(comments)}")
print(f"Số bình luận duy nhất (sau khi lọc trùng): {len(unique_comments)}")

# Nếu bạn muốn chuyển về list để xử lý tiếp (ví dụ lưu file)
final_list = list(unique_comments)

--- THỐNG KÊ ---
Tổng số bình luận quét được: 40
Số bình luận duy nhất (sau khi lọc trùng): 38


In [745]:
for i in unique_comments:
    print(i)

Anh Hoàng trông cũng dễ nhìn, tự lực xây nên cơ nghiệp như giờ, ấm áp. Thì em nào chả mê mà có gì đâu mà lạ. Có mấy con không ăn được anh Hoàng mới chê thôi chứ
Dkm vừa giỏi, vừa giàu, vừa tài hoa mỗi tội hơi xấu trai(kphai hot boy thôi) thì con nào chả muốn bú peter vội. Còn chưa kể gầy là thầy đyt, không phải tự nhiên mà em pháo mồm thì chê piter bé nhưng vẫn cay thì chỉ có cay k dc bú piter thôi :)))
Mồm thì các em chửi đấy, nhưng h a Hoàng  mà nhắn tin đi làm tý bún đậu khéo còn tắm trc mấy ngày để đi vội ý chứ .Tiền tài địa vị nhan sắc cũng méo phải xấu, trai chưa vợ gái chưa chồng mà mấy mẹ cứ sồn sồn lên như cặp với ny mấy mẹ .Nhiều em làm content thì kêu để lọc ck với bạn trai đúng khắm vcl
phấn đào =)))
Tôi thực sự đ hiểu, mặc dù t đ ưa thằng này nhma quan điểm của tôi   ai người đó djt, mình quan tâm làm bu** gì cho mệt. Nói như thằng này cũng chả sai, trai chưa vợ gái chưa chồng, kể cả ny đi nữa thì cũng chả có tí ràng buộc pháp lý nào. Chúng nó là ng nổi tiếng chứ ng thường